In [51]:
import pandas as pd
import numpy as np

In [52]:
# Import data and important features for graph construction
data = pd.read_pickle('../data/connectivity_compliance_matrices.pkl')

# Get only rhos with value greater than 0.2 (rho = 0.3)
data = data.rename(columns={'ρ': 'rho'})
data = data[data['rho'] > 0.2]

# Extract relevant features
X = data[['connectivity_matrix', 'compliance_matrix']]
X = X.reset_index(drop=True)
display(X.head())
print(f"Shape of X: {X.shape}")

,connectivity_matrix,compliance_matrix
0,"[[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...","[[16.2442166588418, -4.390589663678543, -4.392..."
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[22.194443522542244, -7.184492662134119, -7.1..."
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[21.44865815819801, -6.467827160497426, -6.44..."
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[15.247634909853652, -3.931170890685161, -3.9..."
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[23.16273271145176, -5.511611078765939, -5.66..."


Shape of X: (1310, 2)


In [53]:
from torch_geometric.data import Data
import torch_geometric
import torch

In [54]:
# Construct sample graph data consisting of edge index and node features (rho)
sample = X.iloc[0]
connectivity_matrix = sample['connectivity_matrix']
compliance_matrix = sample['compliance_matrix']
edge_index = np.array(np.nonzero(connectivity_matrix)).T
display(sample['connectivity_matrix'])

sample_data = Data(edge_index=torch.tensor(edge_index, dtype=torch.long), y=torch.tensor(compliance_matrix, dtype=torch.float32))
print(f"Edge index:\n{sample_data.edge_index}")
print(f"Compliance matrix:\n{sample_data.y}")

array([[0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.]])

Edge index:
tensor([[ 0,  1],
        [ 0,  4],
        [ 1,  0],
        [ 1,  7],
        [ 2,  3],
        [ 2, 10],
        [ 3,  2],
        [ 3,  4],
        [ 4,  0],
        [ 4,  3],
        [ 4,  6],
        [ 4, 10],
        [ 6,  4],
        [ 6,  6],
        [ 7,  1],
        [ 7,  7],
        [10,  2],
        [10,  4]])
Compliance matrix:
tensor([[16.2442, -4.3906, -4.3921,  0.0000,  0.0000,  0.0000],
        [-4.3906, 16.2539, -4.3922,  0.0000,  0.0000,  0.0000],
        [-4.3921, -4.3922, 16.2602,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, 38.9590,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, 39.0762,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 39.0137]])


In [ ]:
def construct_graph_data(row):
    connectivity_matrix = row['connectivity_matrix']
    compliance_matrix = row['compliance_matrix']
    
    edge_index = np.array(np.nonzero(connectivity_matrix)).T
    num_nodes = connectivity_matrix.shape[0]
    
    # Create dummy node features
    x = torch.ones((num_nodes, 1), dtype=torch.float32)

    graph_data = Data(x=x,
                      edge_index=torch.tensor(edge_index, dtype=torch.long).t().contiguous(), 
                      y=torch.tensor(compliance_matrix.flatten(), dtype=torch.float32))
    return graph_data

# Apply the function to the entire DataFrame to create a list of graph data objects
graph_data_list = X.apply(construct_graph_data, axis=1).tolist()
print(f"Constructed {len(graph_data_list)} graph data objects.")
print(f"Sample graph data object:\n{graph_data_list[0]}")

Constructed 1310 graph data objects.
Sample graph data object:
Data(x=[11, 1], edge_index=[2, 18], y=[36])


In [56]:
from sklearn.model_selection import train_test_split
train_data, foo = train_test_split(graph_data_list, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(foo, test_size=0.5, random_state=42)
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Testing set size: {len(test_data)}")

Training set size: 917
Validation set size: 196
Testing set size: 197


In [57]:
class SimpleGNN(torch.nn.Module):
    def __init__(self, input_dim=1, hidden_dim=16, output_dim=36):
        super(SimpleGNN, self).__init__()
        self.conv1 = torch_geometric.nn.GCNConv(input_dim, hidden_dim)
        self.output_layer = torch.nn.Linear(hidden_dim, output_dim)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # Apply GCN layer
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        
        # Global mean pooling to get graph-level representation
        x = torch_geometric.nn.global_mean_pool(x, batch=None)
        
        # Output layer to predict 36 values (6x6 matrix flattened)
        x = self.output_layer(x)
        
        return x

# Initialize model, loss function, and optimizer
model = SimpleGNN()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"Model: {model}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

Model: SimpleGNN(
  (conv1): GCNConv(1, 16)
  (output_layer): Linear(in_features=16, out_features=36, bias=True)
)
Number of parameters: 644


In [58]:
# Simple training loop
def train_model(model, train_data, val_data, epochs=100):
    model.train()
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # Training
        total_train_loss = 0
        for data in train_data:
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out.squeeze(), data.y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(train_data)
        train_losses.append(avg_train_loss)
        
        # Validation
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for data in val_data:
                out = model(data)
                loss = criterion(out.squeeze(), data.y)
                total_val_loss += loss.item()
        
        avg_val_loss = total_val_loss / len(val_data)
        val_losses.append(avg_val_loss)
        model.train()
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
    
    return train_losses, val_losses

# Train the model
train_losses, val_losses = train_model(model, train_data, val_data, epochs=100)

Epoch 0, Train Loss: 6800017767535.6973, Val Loss: 872.1540
Epoch 10, Train Loss: 6800014876589.4004, Val Loss: 658.3445
Epoch 20, Train Loss: 6800006167847.2148, Val Loss: 765.2523
Epoch 30, Train Loss: 6799993106257.6445, Val Loss: 2530.5351
Epoch 40, Train Loss: 6799975327952.6084, Val Loss: 8057.0421
Epoch 50, Train Loss: 6799952469777.4209, Val Loss: 20167.6389
Epoch 60, Train Loss: 6799924937706.7676, Val Loss: 42399.5029
Epoch 70, Train Loss: 6799892772504.0244, Val Loss: 78999.8727
Epoch 80, Train Loss: 6799855905838.5703, Val Loss: 134902.8015
Epoch 90, Train Loss: 6799814453017.6094, Val Loss: 215730.2573


In [67]:
sample_test_data = test_data[0]
model.eval()
with torch.no_grad():
    prediction = model(sample_test_data)
    print(f"Predicted compliance matrix:\n{prediction.reshape(6,6).round()}")
    print(f"Actual compliance matrix:\n{sample_test_data.y.reshape(6,6)})")

Predicted compliance matrix:
tensor([[1248.,  948., -270.,   -0.,    0.,   -0.],
        [ 948., 1296.,  369.,   -0.,   -0.,    0.],
        [-261.,  374., 1241.,    0.,   -0.,    0.],
        [  -0.,   -0.,    0., 1289.,   -0.,   -0.],
        [   0.,   -0.,   -0.,   -0., 1287.,   -0.],
        [  -0.,    0.,    0.,    0.,    0., 1277.]])
Actual compliance matrix:
tensor([[14.0232, -2.0633, -2.0688,  0.0000,  0.0000,  0.0000],
        [-2.0633, 14.0003, -2.0616,  0.0000,  0.0000,  0.0000],
        [-2.0688, -2.0616, 14.0295,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, 50.7279,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, 50.8233,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 50.6560]]))
